# Introduction to WaPOR and data loading

* **Products used:** WaPOR

## Background
The **Wa**ter **P**roductivity through **O**pen access of **R**emotely sensed derived data (WaPOR) monitors and reports on agricultural water productivity through biophysical measures with a focus on Africa and the Near East. This information assists partner countries improve land and water productivity in both rainfed and irrigated agriculture ([Peiser et al. 2017](https://openknowledge.fao.org/items/cdacf817-c13a-4859-b681-a4bdc20c065c)). 

WaPOR provides numerous datasets related to vegetation productivity and water consumption, and associated meteorological and physical conditions such as soil moisture and precipitation. These datasets can be combined with Digital Earth Africa products, services, and workflows for numerous applications including:
   * Monitoring drought conditions
   * Monitoring the water use efficiency of crops
   * Mapping irrigated areas
   * Estimating crop water requirements
   * Irrigation scheduling and budgeting

## Description
This notebook provides an introduction to WaPOR data and nomenclature, and demonstrates loading and plotting.

1. First, we explore the datasets available and how they are labelled.
2. Then, we download and plot annual evapotranspiration.
3. Finally, we download and plot dekadal (10 day temporal frequency) data.

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Tools` package.

In [ ]:
import datetime
import datacube

from deafrica_tools.load_wapor import get_all_WaPORv3_mapsets, get_WaPORv3_info, load_wapor_ds
from deafrica_tools.plotting import display_map
from wapordl import wapor_map

### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [ ]:
dc = datacube.Datacube(app="WaPOR")

### WaPOR Data

WaPOR data has three levels:
1. Global 300m resolution
2. National 100m resolution
3. Sub-national 20m resolution

The table below covers L1 and L2 datasets. L3 datasets can be viewed in the [WaPOR maps platform](https://data.apps.fao.org/wapor) which is built with the same software as [Digital Earth Africa Maps](https://maps.digitalearth.africa/). L3 datasets cover several regions of interest in northern and eastern Africa. This notebook loads level 3 20m data for Egypt. It is recommended that the [WaPOR maps platform](https://data.apps.fao.org/wapor) is inspected to check the availability of level, variable, and temporal frequency combinations for your area of interest. The maps platform also shows map codes in the data description.

Mapset codes are structured as `level-variable-temporal frequency` as shown below. The temporal frequencies available are:
* A - annual
* M - monthly
* D - dekadal (10 days)

So, for level 3 net primary productivity at dekadal intervals the code would be `L3-NPP-D`.

In [ ]:
get_all_WaPORv3_mapsets()

### Analysis parameters

The cell below specifies:
* The area of interest. This can also be a `.geojson` file
* The folder where the downloaded data will be stored. If you are using this script repeatedly, it is recommended you empty this folder from time to time to manage storage. It is advised that any data required for repeated or future analysis is stored outside the sandbox.
* The variable of interest in the form of a mapset code, explained above.
* The period of interest.

In [ ]:
region =  [31.30, 30.70, 31.40, 30.80] #xmin, ymin, xmax, ymax
folder = "Supplementary_data/WaPOR" # folder that the data will be sent to
variable = "L3-AETI-A" # level-variable-time, see table above
period = ["2018-01-01", "2024-02-01"] # period to load

This demonstration notebook loads an area of cropland in the Nile Delta, Egypt.

In [ ]:
display_map(x=(region[0], region[2]), y=(region[1], region[3]))

### Download netCDF files

This cell downloads the data specified. The `aeti` object is the file path to the stored netCDF. Once this cell is run, netCDF files will be stored in the directory specified above.

In [ ]:
aeti = wapor_map(region, variable, period, folder, extension = '.nc')

### Convert data to xarray

Below, the netCDFs are brought into the analysis environment as xarray datasets and coordinate labels and attributes are assigned. This means the data is in a common format with other Digital Earth Africa products and services, and makes the data easy to interact with. The `load_wapor_ds()` function brings in the data as xarray.

In [ ]:
aeti_xr = load_wapor_ds(filename=aeti, variable=variable)

### Inspect the xarray dataset

Now we have x, y, and time as dimensions and our variable (in this case, actual evapotranspiration and interception) as an xarray dataset. This enables us to easily deal with time and space for analysis.

In [ ]:
aeti_xr

### Plot annual ET

The plots show how AETI varies in space and between years. In the Egypt example, the cropland areas are easily visible as areas with higher AETI.

Note that the scalebar is labelled with information from the WaPOR metadata. This can be accessed by calling `aeti_xr[variable].attrs`, as below, which can be especially useful when checking units for calculation. The `load_wapor_ds()` function takes care of re-scaling when the data is loaded, but it is sensible to check the values are reasonable.

We can also see that the attributes include scale and offset values. These have been incorporated into the `load_wapor_ds()` function so the values returned are in the units shown below, in this case mm/year.

In [ ]:
aeti_xr[variable].attrs

In [ ]:
aeti_xr[variable].plot(col='time', col_wrap=3)

## Load dekadal biomass

The cell below loads dekadal actual evapotranspiration using the same procedure as for annual. The only parameter changed is `variable`.

In [ ]:
variable = 'L3-NPP-D'
period = ["2024-01-01", "2024-03-01"]

npp_d = wapor_map(region, variable, period, folder, extension = '.nc')

npp_d_xr = load_wapor_ds(filename=npp_d, variable=variable)

npp_d_xr

### Plot dekadal net primary productivity

It's interesting to note that in the Egypt example, some areas show very high biomass production > 30t/ha, especially in 2023. This must be considered in the context of several crop cycles occurring within a 12 month period.

In [ ]:
npp_d_xr[variable].plot(col='time', col_wrap=3)

## Conclusion

This notebook demonstrated the range of WaPOR variables available and how to load them in the DE Africa Sandbox environment. Subsequent notebooks will dive deeper into analysing WaPOR data alongside DE Africa data.

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

**Last Tested:**

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')